MIS 285N Cognitive Computing<br>
Final Project<br>
Jerry Che - Jose Guerrero - Riley Moynihan - Noah Placke - Sarah Teng - Palmer Wenzel

# Ingredients Generation Model

Following techniques from:
- https://towardsdatascience.com/generative-adversarial-networks-in-python-73d3972823d3
- https://www.maskaravivek.com/post/gan-synthetic-data-generation/
- (for Wasserstein modifications) https://machinelearningmastery.com/how-to-code-a-wasserstein-generative-adversarial-network-wgan-from-scratch/

#### Read data from CSV.

In [1]:
import pandas as pd
# pd.options.display.max_columns = 500


df = pd.read_csv('../data/kaggle/processed/recipes_processed.csv')#.sample(frac=0.1, random_state=42)

df.head(3)

,name,steps,crabmeat,creamcheese,greenonions,garlicsalt,refrigeratedcrescentdinnerrolls,eggyolk,water,sesameseeds,...,tex-mexseasoning,lightnon-dairywhippedtopping,stelladoroanginetticookies,viennabread,beefroundrumproast,romaineleaf,nuocnam,thaiholybasil,driedblacktrumpetmushrooms,driedwoodearmushrooms
0,crab filled crescent snacks,"heat over to 375 degrees, spray large cookie s...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,curried bean salad,"drain & rinse beans, stir all ingredients toge...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,delicious steak with onion marinade,heat the oil in a heavy-based pan and cook the...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Drop unnecessary columns.

In [2]:
df = df.drop(['name', 'steps'], axis=1)

df.head()

,crabmeat,creamcheese,greenonions,garlicsalt,refrigeratedcrescentdinnerrolls,eggyolk,water,sesameseeds,sweetandsoursauce,garbanzobeans,...,tex-mexseasoning,lightnon-dairywhippedtopping,stelladoroanginetticookies,viennabread,beefroundrumproast,romaineleaf,nuocnam,thaiholybasil,driedblacktrumpetmushrooms,driedwoodearmushrooms
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Drop cols used only once.

In [3]:
df = df.drop([col for col, val in df.sum().iteritems() if val > 75], axis=1)

df.head(3)

,crabmeat,refrigeratedcrescentdinnerrolls,sweetandsoursauce,garbanzobeans,gingerpaste,mildcurrypowder,driedcilantro,creamedcorn,cookedbrownrice,ricecakes,...,tex-mexseasoning,lightnon-dairywhippedtopping,stelladoroanginetticookies,viennabread,beefroundrumproast,romaineleaf,nuocnam,thaiholybasil,driedblacktrumpetmushrooms,driedwoodearmushrooms
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Model Definition

#### Build functions and model definitions.

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout, Input
from tensorflow.keras.optimizers import RMSprop


# Model configs
noise_dim = 1000
dim = 256
data_dim = df.shape[1]


def build_generator():
    generator = Sequential()
    
    generator.add(Dense(dim, input_dim=noise_dim))
    generator.add(Dense(dim, activation='relu'))
    generator.add(Dense(dim * 2, activation='relu'))
    generator.add(Dense(dim * 4, activation='relu'))
    generator.add(Dense(data_dim, activation='tanh'))
    
    generator.compile(loss='binary_crossentropy', optimizer='adam')
    
    return generator


def build_discriminator():
    discriminator = Sequential()
    
    discriminator.add(Dense(dim * 4, input_dim=data_dim))
    discriminator.add(Dropout(0.1))
    discriminator.add(Dense(dim * 2, activation='relu'))
    discriminator.add(Dropout(0.1))
    discriminator.add(Dense(dim, activation='relu'))
    discriminator.add(Dense(1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer='adam')
    
    return discriminator


def build_gan(generator, discriminator):
    # Only train generator in combined model
    discriminator.trainable=False
    
    gan_input = Input(shape=(noise_dim,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    
    # Create the GAN model
    gan = Model(inputs=gan_input, outputs=gan_output)
                      
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    
    return gan


generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000)]            0         
_________________________________________________________________
sequential (Sequential)      (None, 7275)              8435819   
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 8107009   
Total params: 16,542,828
Trainable params: 8,435,819
Non-trainable params: 8,107,009
_________________________________________________________________


#### Create function to display generator output.

In [5]:
import numpy as np


def display_recipe(epoch, generator, examples=1):
    # Create noise
    # noise = np.random.normal(0, 1, size=[examples, noise_dim])
    # noise = np.random.laplace(0, 1, size=[examples, noise_dim])
    noise = random_noise(examples)
    
    # Generate recipes
    generated_recipes = generator.predict(noise)
    
#     # Get used ingredients
#     ingredients = []
#     for i in range(generated_recipes.shape[0]):
#         for j in range(len(generated_recipes[i])):
#             if generated_recipes[i][j] >= 0.25:
                
#                 ingredients.append(df.columns[j])
    
#     # Display
#     print("*** Generated Recipe ***")
#     print(f"# of ingredients: {len(ingredients)}")
#     print(f"Ingredients: {ingredients[:]}")
#     print(generated_recipes[i])

    # Use top N ingredients for recipe
    ingredients = []
    top_ingredients_idx = generated_recipes[0].argsort()[-10:][::-1]
    for idx in top_ingredients_idx:
        ingredients.append(df.columns[idx])
        
    # Display
    print("*** GENERATED RECIPE ***")
    print(f"Ingredients: {ingredients}")

In [6]:
import random


def random_noise(num):
    return np.random.normal(0, 1, size=[num, noise_dim])
    # return np.random.laplace(0, 1, size=[num, noise_dim])
    noise = np.random.laplace(0, 1, size=[num, noise_dim])
    for i in range(len(noise)):
        for j in range(len(noise[i])):
            if random.random() < 0.9:
                noise[i][j] = 0.0
    
    return noise

#### Training logic.

In [7]:
from tqdm import tqdm


def training(df, epochs=1, batch_size=32, sample_interval=10):
    # Get batch count
    batch_count = df.shape[0] / batch_size
    
    # Build GAN
    generator = build_generator()
    discriminator = build_discriminator()
    gan = build_gan(generator, discriminator)
    
    # Training step
    for e in range(1, epochs + 1):
        # for _ in tqdm(range(batch_size)):
            
        # Random noise as an input to initialize the generator
        noise = random_noise(batch_size)
        # replace with Laplace?
        # replace high% of noise with 0

        # Use the GAN to generate "fake" recipes
        generated_recipes = generator.predict(noise)

        # Get a sample of real recipes from data
        # real_recipes = X_train.loc[np.random.randint(low=0, high=X_train.shape[0], size=batch_size)]
        real_recipes = df.sample(batch_size)

        # Mix the real and fake data
        X = np.concatenate([real_recipes, generated_recipes])

        # Create labels for real and fake data
        y_dis = np.zeros(2 * batch_size)  # fake
        y_dis[:batch_size] = 1.0          # real

        # Train the discriminator while generator is fixed
        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(X, y_dis)

        # Fix the images generated by the generator as real
        noise = random_noise(batch_size)
        y_gen = np.ones(batch_size)

        # Train the generator (to have the discriminator label samples as valid)
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gen)

        # Output loss
        print(f"E{e} [D Loss: {d_loss:.4f}] [G loss: {g_loss:.4f}]")
            
        # Display created recipes at a given epoch interval
        if e % sample_interval == 0:
            # Display recipe
            display_recipe(e, generator)
    
    return generator, discriminator, gan


generator, discriminator, gan = training(df, epochs=5000, batch_size=8)

E1 [D Loss: 0.6926] [G loss: 7.3943]
E2 [D Loss: 0.9207] [G loss: 3.3933]
E3 [D Loss: 0.4822] [G loss: 5.9928]
E4 [D Loss: 0.3993] [G loss: 6.7879]
E5 [D Loss: 0.4388] [G loss: 4.0222]
E6 [D Loss: 0.6294] [G loss: 2.0938]
E7 [D Loss: 0.4240] [G loss: 4.7044]
E8 [D Loss: 0.4546] [G loss: 5.0530]
E9 [D Loss: 0.4680] [G loss: 3.9298]
E10 [D Loss: 0.4114] [G loss: 3.7851]
*** GENERATED RECIPE ***
Ingredients: ['butterycrackers', 'italianbrandy', 'refrigeratedcrescentdinnerrolls', 'fatfreesugar-freestrawberrygelatin', 'chocolatecaramel-coveredwafers', 'rindof', 'coconutflavoring', 'oreocookies', 'lightoil', 'orangewedge']
E11 [D Loss: 0.3736] [G loss: 4.4357]
E12 [D Loss: 0.3577] [G loss: 4.0907]
E13 [D Loss: 0.3646] [G loss: 3.3365]
E14 [D Loss: 0.4340] [G loss: 3.3618]
E15 [D Loss: 0.3965] [G loss: 5.2614]
E16 [D Loss: 0.3958] [G loss: 2.9820]
E17 [D Loss: 0.5651] [G loss: 1.5456]
E18 [D Loss: 0.5225] [G loss: 3.0334]
E19 [D Loss: 0.5581] [G loss: 4.1600]
E20 [D Loss: 0.7434] [G loss: 0.6